In [3]:
%matplotlib inline
from __future__ import print_function
import pandas as pd
import sys
sys.path.append('/ref/analysis/pipelines/')
import kang
from collections import Counter 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
def infoparse(x):
    key   = [i.split('=')[0] for i in x.split(';')]
    value = [i.split('=')[1] for i in x.split(';')]
    return dict(zip(key,value))

In [5]:
file_refmap = './predicted/cuffcmp.my_csv.csv.addgene.gff3.sort.gff3.refmap'
file_tmap   = './predicted/cuffcmp.my_csv.csv.addgene.gff3.sort.gff3.tmap'
file_fa     = './Osativa/annotation/Osativa_323_v7.0.cds.fa'
file_pep    = './finalout/my_csv.csv.addgene.gff3.sort.gff3.pep.fa'
file_cds    = './finalout/my_csv.csv.addgene.gff3.sort.gff3.cds.fa'
file_tgff   = './finalout/my_csv.csv.addgene.gff3.sort.gff3.merge.all.gff3.sort.gff3'
file_rgff   = './Osativa/annotation/Osativa_323_v7.0.gene.gff3'

In [20]:
df_rgff      = pd.read_csv(file_rgff,sep='\t',comment='#',header=None)
mask         = (df_rgff[2] == 'mRNA')
df_rgff_mRNA = df_rgff[mask]
df_rgff_mRNA['ID'] = df_rgff_mRNA[8].apply(lambda x: infoparse(x)['ID'])
df_rgff_mRNA['Parent'] = df_rgff_mRNA[8].apply(lambda x : infoparse(x)['Parent'])
df_rgff_mRNA_ix = df_rgff_mRNA.set_index('ID')

df_tgff      = pd.read_csv(file_tgff,sep='\t',comment='#',header=None)
mask         = (df_tgff[2] == 'mRNA')
df_tgff_mRNA = df_tgff[mask]
df_tgff_mRNA['ID'] = df_tgff_mRNA[8].apply(lambda x: infoparse(x)['ID'])
df_tgff_mRNA_ix = df_tgff_mRNA.set_index('ID')

df_rgff      = pd.read_csv(file_rgff,sep='\t',comment='#',header=None)
mask         = (df_rgff[2] == 'CDS')
df_rgff_CDS = df_rgff[mask]
df_rgff_CDS['Parent'] = df_rgff_CDS[8].apply(lambda x: infoparse(x)['Parent'])
df_rgff_CDS_ix = df_rgff_CDS.set_index('Parent')

df_tgff      = pd.read_csv(file_tgff,sep='\t',comment='#',header=None)
mask         = (df_tgff[2] == 'CDS')
df_tgff_CDS = df_tgff[mask]
df_tgff_CDS['Parent'] = df_tgff_CDS[8].apply(lambda x: infoparse(x)['Parent'])
df_tgff_CDS_ix = df_tgff_CDS.set_index('Parent')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [23]:
dict2g = dict(zip(df_rgff_mRNA['ID'],df_rgff_mRNA['Parent']))

In [7]:
def get_posset(df):
    if isinstance(df,pd.Series):
        locs = set(np.arange(df[3],df[4]+1))
    else:
        locs = {0}
        for x,y in df[[3,4]].values:
            locs = locs | set(np.arange(x,y+1))
    return locs - {0}

In [8]:
def matched_genes(chromosome, qpos):
    mask            = (df_rgff_CDS[0] == chromosome) & (df_rgff_CDS[3] < max(qpos) + 100000) & (df_rgff_CDS[3] > min(qpos) - 100000) 
    df_rgff_CDS_sub = df_rgff_CDS[mask].set_index('Parent')
    matched_transcripts = []
    for ix in set(df_rgff_CDS_sub.index):
        edf        = df_rgff_CDS_sub.loc[ix]
        #chromosome = edf[0][0]
        #t_start, t_end = np.min(edf[[3,4]].values), np.max(edf[[3,4]].values)
        #if chromosome == tchr:
        tpos = get_posset(edf)
        if len(qpos & tpos) > 100 :
            matched_transcripts.append(ix)
    return matched_transcripts 

In [25]:
dic = {'qname':[], 
       'merges':[]}
tgff_posset = []
ixs = list(set(df_tgff_CDS_ix.index))
len_dist = []
for ix in tqdm(ixs):
    qpos = get_posset(df_tgff_CDS_ix.loc[ix])
    chromosome  = df_tgff_CDS_ix.loc[ix][0][0]
    matches     = matched_genes(chromosome,qpos)
    matches_set = set([dict2g[x] for x in matches])
    if len(matches_set) > 1:
        #print(ix, matches_set)
        dic['qname'].append(ix)
        dic['merges'].append(matches_set)
        len_dist.append(len(matches_set))
    

100%|██████████| 60657/60657 [34:39<00:00, 29.17it/s]


In [26]:
df_merge = pd.DataFrame(dic)

In [27]:
print("number of predicted transcripts that merge reference transcripts:",len(df_merge))

number of predicted transcripts that merge reference transcripts: 513


In [28]:
sumset = {0}
for i in df_merge['merges']:
    sumset = sumset | i

In [29]:
print("number of reference trancripts that are merged by prediction",len(sumset - {0}))

number of reference trancripts that are merged by prediction 699


In [30]:
df_merge['merged genes'] = df_merge['merges'].apply(lambda x : ','.join(x))

In [31]:
df_merge.to_csv('merged.cases.txt',sep='\t')

In [32]:
df_merge.head()

,merges,qname,merged genes
0,"{LOC_Os02g41820.MSUv7.0, LOC_Os02g41830.MSUv7.0}",Gene.52277::STRG.6536.3::g.52277::m.52277,"LOC_Os02g41820.MSUv7.0,LOC_Os02g41830.MSUv7.0"
1,"{LOC_Os09g07294.MSUv7.0, LOC_Os09g07300.MSUv7.0}",Gene.158203::STRG.24241.1::g.158203::m.158203,"LOC_Os09g07294.MSUv7.0,LOC_Os09g07300.MSUv7.0"
2,"{LOC_Os08g30850.MSUv7.0, LOC_Os08g30870.MSUv7.0}",Gene.146537::STRG.23134.5::g.146537::m.146537,"LOC_Os08g30850.MSUv7.0,LOC_Os08g30870.MSUv7.0"
3,"{LOC_Os12g38420.MSUv7.0, LOC_Os12g38430.MSUv7.0}",Gene.44959::STRG.31557.6::g.44959::m.44959,"LOC_Os12g38420.MSUv7.0,LOC_Os12g38430.MSUv7.0"
4,"{LOC_Os09g15850.MSUv7.0, LOC_Os09g15860.MSUv7.0}",Gene.157505::STRG.24621.4::g.157505::m.157505,"LOC_Os09g15850.MSUv7.0,LOC_Os09g15860.MSUv7.0"


In [17]:
df_tgff_CDS_ix.head()

,0,1,2,3,4,5,6,7,8
Parent,,,,,,,,,
Gene.11756::STRG.1.2::g.11756::m.11756,Chr1,ST/TD/KYJ,CDS,3449,3616,.,+,0,ID=cds.Gene.11756::STRG.1.2::g.11756::m.11756....
Gene.11756::STRG.1.2::g.11756::m.11756,Chr1,ST/TD/KYJ,CDS,4357,4455,.,+,0,ID=cds.Gene.11756::STRG.1.2::g.11756::m.11756....
Gene.11756::STRG.1.2::g.11756::m.11756,Chr1,ST/TD/KYJ,CDS,5457,5560,.,+,0,ID=cds.Gene.11756::STRG.1.2::g.11756::m.11756....
Gene.11756::STRG.1.2::g.11756::m.11756,Chr1,ST/TD/KYJ,CDS,7136,7944,.,+,1,ID=cds.Gene.11756::STRG.1.2::g.11756::m.11756....
Gene.11756::STRG.1.2::g.11756::m.11756,Chr1,ST/TD/KYJ,CDS,8028,8150,.,+,2,ID=cds.Gene.11756::STRG.1.2::g.11756::m.11756....
